In [14]:
import feedparser
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import os
import pymongo

In [15]:
# Get the mongodb password from an environment variable
mongoPass = os.environ['mongoPass']

# Establish the remote connection to the mongo data base
myclient = pymongo.MongoClient("mongodb+srv://axme100:{}@cluster0-5jopz.mongodb.net/test?retryWrites=true&w=majority".format(mongoPass))

# This is the name of the cluster stored on mongo atlas
mydb = myclient["finalProject"]

# Create a new colection called raw article 
mycol = mydb["rawArticles"]

In [16]:
# Define the class that will be created
class rawArticle:
    def __init__(self, url, title, date, publication, articleText):
        self.url = url
        self.title = title
        self.date = date
        self.publication = publication
        self.articleText = articleText
        
    def saveToDatabase(self):
        # Save the entry into the mongo database
        mycol.insert_one({'url': self.url,
                         'title': self.title,
                         'date': self.date,
                         'publication': self.publication,
                         'articleText': self.articleText})

In [17]:
# Define a class of scraper
class scraper:
    
    def scrape_jornada(self):
        # Hard coded URL to be scraped
        url="https://www.jornada.com.mx/rss/edicion.xml?v=1"
        
        # Parse the XML feed
        feedBurner = feedparser.parse(url)
        
        # Loop through all of the entries
        for entry in feedBurner['entries']:
    
            # Get the page that we want to scrape
            html = requests.get(entry['link'])
        
            # Save the page as a beautiful soup object
            bs = BeautifulSoup(html.text, 'html.parser')
    
            articleText = bs.find("div", {"id": "article-text"}).get_text().replace('\n','').strip()
    
            # Create an article object
            myArticle = rawArticle(url=entry['link'],
                          title=entry['title'],
                          date=entry['published'],
                          publication="La Jornada",
                          articleText=articleText)
        
            # Call the object's method to save to a mongo data base
            myArticle.saveToDatabase()
    
    def scrape_artistegui():
        print("hello")

In [18]:
myScrapper = scraper()
myScrapper.scrape_jornada()

AttributeError: 'NoneType' object has no attribute 'get_text'